In [1]:
#Importing necessary libraries
from dotenv import load_dotenv
import os 
from alpha_vantage.timeseries import TimeSeries
import pandas as pd
import hopsworks
import re 
import modal 
#prepocessing
import requests
import pandas as pd
import json
import pandas_market_calendars as mcal
import datetime
import numpy as np
from datetime import datetime, timedelta
load_dotenv()

True

In [2]:
#Connecting to Alpha vantage using API key
api_key = os.environ.get('STOCK_API') # Replace this with your API key
ts = TimeSeries(key=api_key, output_format='pandas')

# Fetch daily adjusted stock prices; adjust the symbol as needed
data, meta_data = ts.get_daily(symbol='NVDA', outputsize='full')

print(data.head())

            1. open  2. high    3. low  4. close    5. volume
date                                                         
2024-08-02   103.76   108.72  101.3700    107.27  482027464.0
2024-08-01   117.53   120.16  106.8104    109.21  523462326.0
2024-07-31   112.90   118.34  110.8800    117.02  473174182.0
2024-07-30   111.52   111.99  102.5400    103.73  486833274.0
2024-07-29   113.69   116.28  111.3000    111.59  248152068.0


In [3]:
#Looking at data info
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6228 entries, 2024-08-02 to 1999-11-01
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   1. open    6228 non-null   float64
 1   2. high    6228 non-null   float64
 2   3. low     6228 non-null   float64
 3   4. close   6228 non-null   float64
 4   5. volume  6228 non-null   float64
dtypes: float64(5)
memory usage: 291.9 KB


In [4]:
#Looking at the meta data
meta_data

{'1. Information': 'Daily Prices (open, high, low, close) and Volumes',
 '2. Symbol': 'NVDA',
 '3. Last Refreshed': '2024-08-02',
 '4. Output Size': 'Full size',
 '5. Time Zone': 'US/Eastern'}

In [5]:
#Stock market:
def today_is_a_business_day(today):
    # Get the NYSE calendar
    cal = mcal.get_calendar('NYSE')
    schedule = cal.schedule(start_date=today, end_date=today) # Get the NYSE calendar's open and close times for the specified period
    try:
        isBusinessDay = schedule.market_open.dt.strftime('%Y-%m-%d')
        return True
    except:
        print('Today {} is not a business day'.format(today))
        return False

In [6]:
#Defining a function to find the next business day
def next_business_day(today):
    
    # Real tomorrow
    tomorrow = (today + timedelta(days=1)).strftime("%Y-%m-%d")
    
    # Get the NYSE calendar
    cal = mcal.get_calendar('NYSE')

    found_next_business_day = False 
    while not found_next_business_day:
        schedule = cal.schedule(start_date=tomorrow, end_date=tomorrow) # Get the NYSE calendar's open and close times for the specified period
        try:
            isBusinessDay = schedule.market_open.dt.strftime('%Y-%m-%d') # Only need a list of dates when it's open (not open and close times)
            found_next_business_day = True
        except:
            print('The date {} is not a business day'.format(tomorrow))
            tomorrow = (datetime.datetime.strptime(tomorrow,"%Y-%m-%d") + timedelta(days=1)).strftime("%Y-%m-%d")
            
    return isBusinessDay.to_numpy()[0]

In [7]:
#Defining a function to extract business day
def extract_business_day(start_date,end_date):
    """
    Given a start_date and end_date.
    
    `Returns`:
    
    isBusinessDay: list of str (with all dates being business days)
    is_open: boolean list
        e.g is_open = [1,0,...,1] means that start_date = open, day after start_date = closed, and end_date = open
    """
    
    # Saving for later
    end_date_save = end_date
    
    # Getting the NYSE calendar
    cal = mcal.get_calendar('NYSE')

    # Getting the NYSE calendar's open and close times for the specified period
    schedule = cal.schedule(start_date=start_date, end_date=end_date)
    
    # Only need a list of dates when it's open (not open and close times)
    isBusinessDay = np.array(schedule.market_open.dt.strftime('%Y-%m-%d')) 
    
    # Going over all days: 
    delta = datetime.timedelta(days=1)
    start_date = datetime.datetime.strptime(start_date,"%Y-%m-%d") #datetime.date(2015, 7, 16)
    end_date = datetime.datetime.strptime(end_date,"%Y-%m-%d") #datetime.date(2023, 1, 4)
    
    # Extracting days from the timedelta object
    num_days = (end_date - start_date).days + 1
    
    # Creating a boolean array for days being open (1) and closed (0) 
    is_open = np.zeros(num_days)
    
    # iterate over range of dates
    current_BusinessDay = isBusinessDay[0]
    count_dates = 0
    next_BusinessDay = 0
    
    while (start_date <= end_date):
    
        if start_date.strftime('%Y-%m-%d') == current_BusinessDay:
            is_open[count_dates] = True

            if current_BusinessDay == end_date_save or current_BusinessDay==isBusinessDay[-1]:
                break
            else:
                next_BusinessDay += 1
                current_BusinessDay = isBusinessDay[next_BusinessDay]
        else:
            is_open[count_dates] = False

        count_dates += 1   
        start_date += delta
        
    print(np.shape(is_open))
        
    return isBusinessDay, is_open

In [8]:
#Defining a function to clean the column names
def clean_column_name(name):
    # Remove all non-letter characters
    cleaned_name = re.sub(r'[^a-zA-Z]', '', name)
    return cleaned_name

In [9]:
data.columns = [clean_column_name(col) for col in data.columns]

In [10]:
data.head()

,open,high,low,close,volume
date,,,,,
2024-08-02,103.76,108.72,101.3700,107.27,482027464.0
2024-08-01,117.53,120.16,106.8104,109.21,523462326.0
2024-07-31,112.90,118.34,110.8800,117.02,473174182.0
2024-07-30,111.52,111.99,102.5400,103.73,486833274.0
2024-07-29,113.69,116.28,111.3000,111.59,248152068.0


In [11]:
data.reset_index(inplace=True)

In [12]:
data.head()

,date,open,high,low,close,volume
0,2024-08-02,103.76,108.72,101.3700,107.27,482027464.0
1,2024-08-01,117.53,120.16,106.8104,109.21,523462326.0
2,2024-07-31,112.90,118.34,110.8800,117.02,473174182.0
3,2024-07-30,111.52,111.99,102.5400,103.73,486833274.0
4,2024-07-29,113.69,116.28,111.3000,111.59,248152068.0


In [13]:
# Define the date range we're interested in
yesterday =datetime.now()-timedelta(days=1)
two_years_back = yesterday - timedelta(days=684)

In [14]:
# Filtering the DataFrame to this range
filtered_df = data[(data['date'] >= two_years_back) & (data['date'] <= yesterday)]

In [15]:
filtered_df.head()

,date,open,high,low,close,volume
0,2024-08-02,103.76,108.72,101.3700,107.27,482027464.0
1,2024-08-01,117.53,120.16,106.8104,109.21,523462326.0
2,2024-07-31,112.90,118.34,110.8800,117.02,473174182.0
3,2024-07-30,111.52,111.99,102.5400,103.73,486833274.0
4,2024-07-29,113.69,116.28,111.3000,111.59,248152068.0


In [16]:
print(filtered_df['date'].min())
print(filtered_df['date'].max())

2022-09-21 00:00:00
2024-08-02 00:00:00


In [17]:
filtered_df.shape

(469, 6)